# Survival Analysis in International Relations 
## Replication Code for Tables in Chapter

We need the following packages to run this notebook:
```R
install.packages(c("haven", "stargazer", "survival","ggplot2", "ggrepel", "easypackages", "coxed", "flexsurv", 'eha', "survminer"))
```

In [ ]:
easypackages::libraries("haven", "stargazer", "survival","ggplot2", "eha", "coxed", "dplyr", "flexsurv", "survminer")
options(scipen = 999)

#### Table 1: Model Fit Comparison of Findley and Young (2015)

Package `eha` does not include `exponential` distribution. Exponential model can be obtained by choosing `weibull` in combination with 'shape=1'

In [ ]:
duration <- haven::read_dta('replication-data/duration_main_est.dta')
duration <- duration[duration$`_st` == 1,]

duration$start_date <- duration$`_t0`
duration$end_date <- duration$`_t`

duration <- as.data.frame(duration)

In [27]:
model_normal = aftreg(Surv(start_date, end_date, warend) ~ lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+mountains+guarantee, 
                       data = duration, dist = 'lognormal'); 

model_weibull = aftreg(Surv(start_date, end_date, warend) ~ lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+mountains+guarantee, 
                          data = duration, dist = 'weibull')

model_log = aftreg(Surv(start_date, end_date, warend) ~ lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+mountains+guarantee, 
                          data = duration, dist = 'loglogistic')

model_exp = aftreg(Surv(start_date, end_date, warend) ~ lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+mountains+guarantee, 
                          data = duration, dist = 'weibull', shape =1)


In [ ]:
model_fit <- cbind(model_exp$loglik[2], model_weibull$loglik[2], model_normal$loglik[2], model_log$loglik[2])

rownames(model_fit) = c("Log-Likelihood")
colnames(model_fit) = c("Exponential", "Weibull", "Log Normal", "Log Logistic")

stargazer(model_fit, digits = 1, title ="Model Fit Comparison of Findley and Young (2015)", flip=TRUE, notes.align ="c", type='text')

We now compare the performance of models using `flexsurv` package 

In [ ]:
flex_normal = flexsurv::flexsurvreg(Surv(start_date, end_date, warend) ~ lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+mountains+guarantee, 
                       data = duration, dist = 'lognormal'); 

In [ ]:
flex_weibull = flexsurv::flexsurvreg(Surv(start_date, end_date, warend) ~ lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+mountains+guarantee, 
                          data = duration, dist = 'weibull')

In [ ]:
flex_log = flexsurv::flexsurvreg(Surv(start_date, end_date, warend) ~ lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+mountains+guarantee, 
                          data = duration, dist = 'llogis')

In [ ]:
flex_exp = flexsurv::flexsurvreg(Surv(start_date, end_date, warend) ~ lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+mountains+guarantee, 
                          data = duration, dist = 'exponential')

In [ ]:
dist <- list("lognormal", "weibull", "llogis", "exponential")

survival.function <- function(dist){
    model <- flexsurv::flexsurvreg(Surv(start_date, end_date, warend) ~ lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+mountains+guarantee, 
                       data = duration, dist = dist)
    aic <- model$AIC
    loglik <- model$loglik
    score <- rbind(aic, loglik)
}

model_fit <- sapply(dist, FUN = survival.function, USE.NAMES = TRUE)

In [ ]:
rownames(model_fit) = c("AIC", "Log-Likelihood")

colnames(model_fit) = c("Log Normal", "Weibull", "Log Logistic", "Exponential")

stargazer(model_fit, type ='text', flip = TRUE, digits = 1, title ="Model Fit Comparison of Findley and Young (2015)")

#### Table 3: Survival Modeling of Civil War Duration (Findley and Young 2015)



The default parameterization in R is the accelerated failure time (A.F.T). For weibull-PH model, we use `eha` package in R which produces results similar to streg in Stata. 

In [23]:
model_normal = aftreg(Surv(start_date, end_date, warend) ~ lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+mountains+guarantee, 
                       data = duration, dist = 'lognormal')

weibulAFT = aftreg(Surv(start_date, end_date, warend) ~ lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+mountains+guarantee, 
                          data = duration, dist = 'weibull')

weibull_ph = phreg(Surv(start_date, end_date, warend) ~ lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+mountains+guarantee, 
              dist= 'weibull', data = duration)

cox_model = coxreg(Surv(start_date, end_date, warend) ~ lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+
             mountains+guarantee,data = duration, method="breslow")

Unlike `stcox` function in Stata, `coxph` function from `survival` package in R only reports exponentiated coefficients (hazard ratio), but not adjusted standard errors. In order to fix this problem, we need to first adjust the errors by the ratio of the number of groups in the clustered errors and also scale them using the delta method. So we fit a second Cox model and adjust the standard errors. However, `Stargazer` only accepts numeric values, so we convert `se_cox` from list to numeric to include in stargazer

In [31]:
Cox_formula <- as.formula(Surv(start_date, end_date, warend) ~ lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+mountains+guarantee, 
                  cluster(warnumber))
Cox_HR <- coxph(Cox_formula, data = duration, method="breslow")

adjustment <- nlevels(factor(duration$warnumber))
se_cox <- c()
se_cox[[1]] <- sqrt(exp(coef(Cox_HR))^2*diag(vcov(Cox_HR)) * (adjustment/(adjustment-1)))
Cox_HR$coefficients <- exp(Cox_HR$coefficients)

tval <- c()
tval[[1]] <- as.vector(Cox_HR$coefficients)/se_cox[[1]]
pvalues <- c()
pvalues[[1]] <- pt(abs(tval[[1]]), df = 10, lower.tail = FALSE)*2

se_cox = as.numeric(as.character(unlist(se_cox[[1]])))

In [ ]:
stargazer(weibull_ph, weibulAFT, model_normal, cox_model, Cox_HR,
          se = list(NULL, NULL, NULL, NULL, se_cox), 
          covariate.labels = c('Terrorism(log/lag)', 'Population(log)','Ethnic fractionalization','GDP(log)', 'Number of actors',
                               'Battle deaths (log)', 'Mountainous terrain', 'Security guarantee'), 
          column.labels = c('Weibul(PH)', 'Weibul(AFT)', 'Log Normal', 'Cox Model', 'Cox Model (HR)'), 
          dep.var.labels.include = FALSE, 
          dep.var.caption="", 
          keep.stat = c('aic', 'll'), 
          out = 'all.txt',
          model.names = FALSE, type= 'text', 
          star.cutoffs=NA, 
          notes='p < 0.05; Standard errors reported in parentheses', notes.append = FALSE) 


#### Table 4: Time Varying Test of Proportional Hazards

The results of three models are presented in the table; a cox model with all covariates, a model with an interaction term between terrorism and time (`lagLogTotalWarRelated:warmonths`), and a model with an interaction term that looks at wars longer the 81 months (the median value). As explained earlier, we need to transform standard errors of `coxph` to obtain standard errors for exponentiated coefficients. 

In [110]:
formula0 <- as.formula(Surv(start_date, end_date, warend) ~ 
                       lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+mountains+guarantee, cluster(warnumber))
cox_ph <- coxph(formula0, data = duration, method="breslow")

adjustment <- nlevels(factor(duration$warnumber))
se_cox <- c()
se_cox[[1]] <- sqrt(exp(coef(cox_ph))^2*diag(vcov(cox_ph)) * (adjustment/(adjustment-1)))
cox_ph$coefficients <- exp(cox_ph$coefficients)

tval <- c()
tval[[1]] <- as.vector(cox_ph$coefficients)/se_cox[[1]]
pvalues <- c()
pvalues[[1]] <- pt(abs(tval[[1]]), df = 10, lower.tail = FALSE)*2

se_cox = as.numeric(as.character(unlist(se_cox[[1]])))

In [111]:
formula1 <- as.formula(Surv(start_date, end_date, warend) ~ lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+mountains+guarantee+
                      lagLogTotalWarRelated:warmonths, cluster(warnumber))
tvc_cox = coxph(formula1, data = duration, method= 'breslow')

adjustment <- nlevels(factor(duration$warnumber))
se_coxt <- c()
se_coxt[[1]] <- sqrt(exp(coef(tvc_cox))^2*diag(vcov(tvc_cox)) * (adjustment/(adjustment-1)))
tvc_cox$coefficients <- exp(tvc_cox$coefficients)

tval <- c()
tval[[1]] <- as.vector(tvc_cox$coefficients)/se_coxt[[1]]
pvalues <- c()
pvalues[[1]] <- pt(abs(tval[[1]]), df = 10, lower.tail = FALSE)*2

se_coxt = as.numeric(as.character(unlist(se_coxt[[1]])))

To create model 3, we use th time transforme functionality of `coxph` to examine the possibility that terror events have more effect on the duration of civil wars later in campaigns

In [108]:
formula2 <- as.formula(Surv(start_date, end_date, warend) ~ 
                      lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+mountains+guarantee+tt(lagLogTotalWarRelated),cluster(warnumber))
tvc_cox2 = coxph(formula2, tt=function(x, t, ...) ifelse(t > 81, x, 0), data = duration, method= 'breslow')
                 
adjustment <- nlevels(factor(duration$warnumber))
se_coxt2 <- c()
se_coxt2[[1]] <- sqrt(exp(coef(tvc_cox2))^2*diag(vcov(tvc_cox2)) * (adjustment/(adjustment-1)))
tvc_cox2$coefficients <- exp(tvc_cox2$coefficients)

tval <- c()
tval[[1]] <- as.vector(tvc_cox2$coefficients)/se_coxt2[[1]]
pvalues <- c()
pvalues[[1]] <- pt(abs(tval[[1]]), df = 10, lower.tail = FALSE)*2

se_coxt2 = as.numeric(as.character(unlist(se_coxt2[[1]])))

In [113]:
stargazer(cox_ph, tvc_cox, tvc_cox2, se = list(se_cox, se_coxt, se_coxt2), column.labels = c("Cox (PH)", "Cox TVC", "Cox TVC"), 
          title='Time Varying Test of Proportional Hazards', dep.var.labels.include = FALSE, p.auto=FALSE, dep.var.caption="",
          covariate.labels = c("Terrorism(log/lag)","Terrorism(log/lag):Duration", "Terrorism(log/lag):Duration"), omit.table.layout = "sn",
          keep = c("lagLogTotalWarRelated", "lagLogTotalWarRelated:warmonths"),
          omit.stat=c("LL","ser","f", "wald", "rsq", "lr", "logrank", "max.rsq"), star.cutoffs=NA,
          model.names = FALSE, type='text', out='model_output/time_varying_cox.txt')


Time Varying Test of Proportional Hazards
                            Cox (PH) Cox TVC Cox TVC
                              (1)      (2)     (3)  
----------------------------------------------------
Terrorism(log/lag)           0.741    0.785   0.696 
                            (0.123)  (0.170) (0.153)
                                                    
Terrorism(log/lag):Duration           0.999         
                                     (0.002)        
                                                    
Terrorism(log/lag):Duration                   1.149 
                                             (0.348)
                                                    


#### Table 5: Schoenfeld Residual Test of Proportional Hazards

In [ ]:
formula <- as.formula(Surv(start_date, end_date, warend) ~ lagLogTotalWarRelated+logpop+elf+lngdp+uppsalaMaxed+logbattledeaths+mountains+guarantee)
coxmodel <- coxph(formula, data = duration, method="breslow")

adjustment <- nlevels(factor(duration$warnumber))
se_cox <- c()
se_cox[[1]] <- sqrt(exp(coef(coxmodel))^2*diag(vcov(coxmodel)) * (adjustment/(adjustment-1)))
coxmodel$coefficients <- exp(coxmodel$coefficients)

tval <- c()
tval[[1]] <- as.vector(coxmodel$coefficients)/se_cox[[1]]
pvalues <- c()
pvalues[[1]] <- pt(abs(tval[[1]]), df = 10, lower.tail = FALSE)*2

temp <- (cox.zph(coxmodel))
print(temp)